# Editing Software

In [22]:
import json

import duckdb
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import util
from IPython.display import HTML

pio.renderers.default = "plotly_mimetype"
duckdb.execute("SET enable_progress_bar = false")
pd.set_option("display.max_rows", 100)

default_layout = dict(
    margin=dict(l=55, r=55, b=55, t=55),
    font=dict(family="Times", size=15),
    title_x=0.5,
    paper_bgcolor="#f5f2f0",
    plot_bgcolor="#f5f2f0",
    xaxis=dict(tickcolor="black", linecolor="black", showgrid=True, gridcolor="darkgray", zerolinecolor="darkgray"),
    yaxis=dict(
        tickcolor="black",
        linecolor="black",
        showgrid=True,
        gridcolor="darkgray",
        zerolinecolor="darkgray",
        rangemode="tozero",
    ),
)

display(
    HTML(
        '<link rel="stylesheet" type="text/css" href="../notebooks/notebook.css"><script src="../notebooks/notebook.js"></script>'
    )
)

## Top 10 Editing Software

In [23]:
# Get top 10 editing software by total contributors
top_10_software = (
    duckdb.sql("""
WITH software_total_contributors AS (
    SELECT
        created_by,
        COUNT(DISTINCT user_name) as total_contributors
    FROM '../data_enriched/year=*/month=*/*.parquet'
    WHERE created_by IS NOT NULL
    GROUP BY created_by
    ORDER BY total_contributors DESC
    LIMIT 10
)
SELECT created_by FROM software_total_contributors
""")
    .df()["created_by"]
    .tolist()
)

# Monthly contributors for top 10 editing software
df_monthly_contributors_by_software = duckdb.sql(f"""
SELECT 
    CONCAT(year, '-', LPAD(CAST(month as VARCHAR), 2, '0')) as months,
    created_by,
    COUNT(DISTINCT user_name) as contributors
FROM '../data_enriched/year=*/month=*/*.parquet'
WHERE created_by IN ({",".join([f"'{s}'" for s in top_10_software])})
GROUP BY year, month, created_by
ORDER BY year, month, created_by
""").df()[:-10]  # Remove last month for each software (incomplete data)

# Monthly new contributors for top 10 editing software
df_monthly_new_contributors_by_software = duckdb.sql(f"""
WITH user_first_appearance AS (
    SELECT 
        user_name,
        year,
        month,
        created_by,
        ROW_NUMBER() OVER (PARTITION BY user_name ORDER BY year, month) as rn
    FROM (
        SELECT DISTINCT user_name, year, month, created_by
        FROM '../data_enriched/year=*/month=*/*.parquet'
        WHERE created_by IN ({",".join([f"'{s}'" for s in top_10_software])})
    )
),
first_appearances AS (
    SELECT user_name, year, month, created_by
    FROM user_first_appearance 
    WHERE rn = 1
)
SELECT
    CONCAT(year, '-', LPAD(CAST(month as VARCHAR), 2, '0')) as months,
    created_by,
    COUNT(DISTINCT user_name) as new_contributors
FROM first_appearances
GROUP BY year, month, created_by
ORDER BY year, month, created_by
""").df()[:-10]

# Create the plot
fig = go.Figure()
buttons = []

# Add traces for all contributors (initially visible)
for software in top_10_software:
    software_data = df_monthly_contributors_by_software[df_monthly_contributors_by_software["created_by"] == software]
    fig.add_trace(
        go.Scatter(
            x=software_data["months"],
            y=software_data["contributors"],
            name=software,
            visible=True,
            hovertemplate=f"{software}" + "<br>%{x}<br>%{y:,} contributors<extra></extra>",
        )
    )

# Add traces for new contributors (initially hidden)
for software in top_10_software:
    software_data = df_monthly_new_contributors_by_software[
        df_monthly_new_contributors_by_software["created_by"] == software
    ]
    fig.add_trace(
        go.Scatter(
            x=software_data["months"],
            y=software_data["new_contributors"],
            name=software,
            visible=False,
            hovertemplate=f"{software}" + "<br>%{x}<br>%{y:,} new contributors<extra></extra>",
        )
    )

# Create buttons for switching between all and new contributors
buttons.append(
    {
        "label": "All Contributors",
        "args": [
            {"visible": [True] * len(top_10_software) + [False] * len(top_10_software)},
            {"title.text": "Monthly Contributors by Top 10 Editing Software"},
        ],
        "method": "update",
    }
)

buttons.append(
    {
        "label": "New Contributors",
        "args": [
            {"visible": [False] * len(top_10_software) + [True] * len(top_10_software)},
            {"title.text": "Monthly New Contributors by Top 10 Editing Software"},
        ],
        "method": "update",
    }
)

fig.update_layout(
    title="Monthly Contributors by Top 10 Editing Software",
    xaxis_title="Month",
    yaxis_title="Number of Contributors",
    updatemenus=[{"type": "buttons", "buttons": buttons}],
    **default_layout,
)

fig.show()

## Top 100 Editing Software by Yearly Edit Count

In [53]:
# Edit Count
df_edit_count = duckdb.sql("""
WITH software_yearly_edits AS (
	SELECT
		year,
		created_by as "Editing Software",
		CAST(SUM(edit_count) as BIGINT) as edits
	FROM '../data_enriched/year=*/month=*/*.parquet'
	WHERE created_by IS NOT NULL
	GROUP BY year, created_by
),
software_total_edits AS (
	SELECT
		"Editing Software",
		CAST(SUM(edits) as BIGINT) as total_edits
	FROM software_yearly_edits
	GROUP BY "Editing Software"
	ORDER BY total_edits DESC
	LIMIT 100
)
SELECT
	year,
	"Editing Software",
	edits
FROM software_yearly_edits
WHERE "Editing Software" IN (SELECT "Editing Software" FROM software_total_edits)
ORDER BY year, "Editing Software"
""").df()
df_edit_count = df_edit_count.pivot_table(index="Editing Software", columns="year", values="edits").fillna(0)
df_edit_count["Total"] = df_edit_count.sum(axis=1)
df_edit_count = df_edit_count.sort_values("Total", ascending=False).reset_index()
df_edit_count.insert(0, "Rank", range(1, len(df_edit_count) + 1))
df_edit_count.columns.name = None

# Contributors Count
df_contributors_count = duckdb.sql("""
WITH software_yearly_contributors AS (
	SELECT
		year,
		created_by as "Editing Software",
		CAST(COUNT(DISTINCT user_name) as BIGINT) as contributors
	FROM '../data_enriched/year=*/month=*/*.parquet'
	WHERE created_by IS NOT NULL
	GROUP BY year, created_by
),
software_total_unique_contributors AS (
	SELECT
		created_by as "Editing Software",
		CAST(COUNT(DISTINCT user_name) as BIGINT) as total_contributors
	FROM '../data_enriched/year=*/month=*/*.parquet'
	WHERE created_by IS NOT NULL
	GROUP BY created_by
	ORDER BY total_contributors DESC
	LIMIT 100
)
SELECT
	year,
	"Editing Software",
	contributors
FROM software_yearly_contributors
WHERE "Editing Software" IN (SELECT "Editing Software" FROM software_total_unique_contributors)
ORDER BY year, "Editing Software"
""").df()
df_contributors_count = df_contributors_count.pivot_table(
    index="Editing Software", columns="year", values="contributors"
).fillna(0)

df_contributors_total = duckdb.sql("""
SELECT
	created_by as "Editing Software",
	CAST(COUNT(DISTINCT user_name) as BIGINT) as total_contributors
FROM '../data_enriched/year=*/month=*/*.parquet'
WHERE created_by IS NOT NULL
	AND created_by IN (
		SELECT created_by 
		FROM (
			SELECT created_by, COUNT(DISTINCT user_name) as cnt
			FROM '../data_enriched/year=*/month=*/*.parquet'
			WHERE created_by IS NOT NULL
			GROUP BY created_by
			ORDER BY cnt DESC
			LIMIT 100
		)
	)
GROUP BY created_by
ORDER BY total_contributors DESC
""").df()
df_contributors_count = df_contributors_count.reset_index()
df_contributors_count = df_contributors_count.merge(
    df_contributors_total.set_index("Editing Software")["total_contributors"].rename("Total"),
    left_on="Editing Software",
    right_index=True,
    how="left",
)
df_contributors_count = df_contributors_count.sort_values("Total", ascending=False).reset_index(drop=True)
df_contributors_count.insert(0, "Rank", range(1, len(df_contributors_count) + 1))
df_contributors_count.columns.name = None


with open("../config/replace_rules_created_by.json") as f:
    name_to_html_link = {
        name: f'<a href="{item["link"]}">{name}</a>' for name, item in json.load(f).items() if "link" in item
    }

for df in [df_edit_count, df_contributors_count]:
    df["Editing Software"] = df["Editing Software"].apply(
        lambda name: name_to_html_link[name] if name in name_to_html_link else name
    )

recent_columns = ["Rank", "Editing Software", *df_edit_count.columns[-5:]]
tables = {
    "Edit Count": df_edit_count[recent_columns],
    "Edit Count All Years": df_edit_count,
    "Contributors Count": df_contributors_count[recent_columns],
    "Contributors Count All Years": df_contributors_count,
}

display(HTML(util.get_tables_html(tables, center_columns=["Rank", "Editing Software"])))

Rank,Editing Software,2022,2023,2024,2025,Total
1,JOSM,"758,566,474","756,759,812","767,135,437","325,128,093","10,906,315,113"
2,iD,"416,745,824","427,963,559","428,165,596","191,281,367","3,839,865,430"
3,Potlatch,"2,678,828","2,574,667","2,766,754","977,964","783,200,809"
4,Rapid,"63,564,994","63,683,830","52,467,053","20,156,724","327,340,538"
5,osmtools,"12,399,003","29,128,241","2,154,313","578,476","260,066,827"
6,bulk_upload.py,0,0,"4,341",0,"121,557,139"
7,StreetComplete,"12,904,369","13,630,403","12,782,820","4,729,050","65,361,647"
8,Merkaartor,"491,775","123,828","96,656","150,822","59,742,307"
9,upload.py,"822,536","968,637","2,536,270","699,623","57,155,171"
10,Vespucci,"3,864,897","4,998,545","9,855,980","5,241,579","40,229,893"
